## Definiere einen einfachen WHILE-Parser
(als PEG (parsing expression grammar), die ist sehr ähnlich, aber etwas leistungsfähiger als LL/LR und steht über das `pyparsing` modul zur Verfügung)

Erst einmal den Lexing-Teil:

In [1]:
from pyparsing import Literal, Group, Forward, Word, alphas, nums, Combine, delimitedList

ASSIGN         =  Literal(":=")
LT             =  Literal("<")
INCREMENT      =  Literal("++")
SEMICOLON      =  Literal(";").suppress()
WHILE          =  Literal("WHILE")
LOOP           =  Literal("LOOP")
DO             =  Literal("DO").suppress()
OD             =  Literal("OD").suppress()
IF             =  Literal("IF")
FI             =  Literal("FI").suppress()
THEN           =  Literal("THEN").suppress()
ELSE           =  Literal("ELSE").suppress()
ZERO           =  Literal("0")
VAR_PREFIX     =  Literal("x")

Die folgenden Definitionen werden rekursiv, daher ist folgendes vorher notwendig: 

In [2]:
program        =  Forward()
statement      =  Forward()
conditional    =  Forward()
conditional2   =  Forward()
while_loop     =  Forward()
loop_loop      =  Forward()

und nun können wir den eigentlichen Parser definieren:

In [3]:
var            =  Combine(VAR_PREFIX+Word(nums))
assignment     =  Group(var + ASSIGN + var)
comparison     =  Group(var + LT + var)
inkrement      =  Group(var + INCREMENT)
zeroing        =  Group(var + ASSIGN + "0")
conditional    << (Group(IF + comparison + THEN + Group(program) +ELSE +  Group(program)+ FI))
conditional2    << (Group(IF + comparison + THEN + Group(program) + FI))
while_loop     << (Group(WHILE + comparison + DO + Group(program) + OD))
loop_loop      << (Group(LOOP + var + DO + Group(program) + OD))
statement      << (while_loop | loop_loop |  assignment | inkrement | zeroing | conditional2 | conditional)
program        << delimitedList(statement,delim=";")

Forward: Forward: {{{{{{Forward: Group:({{{{"WHILE" Group:({{Combine:({"x" W:(0123...)}) "<"} Combine:({"x" W:(0123...)})})} Suppress:("DO")} Group:(Forward: None)} Suppress:("OD")}) | Forward: Group:({{{{"LOOP" Combine:({"x" W:(0123...)})} Suppress:("DO")} Group:(Forward: None)} Suppress:("OD")})} | Group:({{Combine:({"x" W:(0123...)}) ":="} Combine:({"x" W:(0123...)})})} | Group:({Combine:({"x" W:(0123...)}) "++"})} | Group:({{Combine:({"x" W:(0123...)}) ":="} "0"})} | Forward: Group:({{{{"IF" Group:({{Combine:({"x" W:(0123...)}) "<"} Combine:({"x" W:(0123...)})})} Suppress:("THEN")} Group:(Forward: None)} Suppress:("FI")})} | Forward: Group:({{{{{{"IF" Group:({{Combine:({"x" W:(0123...)}) "<"} Combine:({"x" W:(0123...)})})} Suppress:("THEN")} Group:(Forward: None)} Suppress:("ELSE")} Group:(Forward: None)} Suppress:("FI")})} [; Forward: {{{{{{Forward: Group:({{{{"WHILE" Group:({{Combine:({"x" W:(0123...)}) "<"} Combine:({"x" W:(0123...)})})} Suppress:("DO")} Group:(Forward: None)} S

## ... nun können wir den Parser mit ein paar einfachen Programmen testen

In [4]:
for prg_str in ("x1:=x0",
                   "x1++",
                   "x1:=0",
                   "x1:=x0;x0++",
                   "WHILE x0<x1 DO x0:=x0 OD",
                   "IF x0<x1 THEN x0:=x0 FI",
                   "IF x0<x1 THEN x0:=x0 ELSE x1:=x1 FI",
                   "LOOP x0 DO x0:=x0 OD",
                   "x1:=x123; WHILE x0<x1 DO x0:=x0; x1:=x1 OD; x0++",
                   "x1:=x123; LOOP x0 DO x0:=x0; x1:=x1 OD; x0++",
                   "x0:=x1; LOOP x2 DO x0:=x0; x1:=x1; WHILE x1<x2 DO x1:=x1 OD OD"
                  ) :
    prg = program.parseString(prg_str, parseAll=True)
    print(prg)

[['x1', ':=', 'x0']]
[['x1', '++']]
[['x1', ':=', '0']]
[['x1', ':=', 'x0'], ['x0', '++']]
[['WHILE', ['x0', '<', 'x1'], [['x0', ':=', 'x0']]]]
[['IF', ['x0', '<', 'x1'], [['x0', ':=', 'x0']]]]
[['IF', ['x0', '<', 'x1'], [['x0', ':=', 'x0']], [['x1', ':=', 'x1']]]]
[['LOOP', 'x0', [['x0', ':=', 'x0']]]]
[['x1', ':=', 'x123'], ['WHILE', ['x0', '<', 'x1'], [['x0', ':=', 'x0'], ['x1', ':=', 'x1']]], ['x0', '++']]
[['x1', ':=', 'x123'], ['LOOP', 'x0', [['x0', ':=', 'x0'], ['x1', ':=', 'x1']]], ['x0', '++']]
[['x0', ':=', 'x1'], ['LOOP', 'x2', [['x0', ':=', 'x0'], ['x1', ':=', 'x1'], ['WHILE', ['x1', '<', 'x2'], [['x1', ':=', 'x1']]]]]]


### Für jede Art von (top-level) Programmstruktur definieren wir ein Prädikat:

In [5]:
def isSeq(p)        : return type(p[0]) is list
def isEmptySeq(p)   : return len(p)==0
def isZeroing(p)    : return p[1]==":=" and p[2]=="0"
def isAssignment(p) : return p[1]==":="
def isIncrement(p)  : return p[1]=="++"
def isWhile(p)      : return p[0]=='WHILE'
def isLoop(p)       : return p[0]=='LOOP'
def isConditional(p): return p[0]=='IF'
def hasElse(p)      : return isConditional(p) and len(p)==4

### Nun können wir einen WHILE-Interpreter implementieren (auf Basis der Semantik)

In [18]:
def interpret(p,b):
    if   isEmptySeq(p)  : return b
    elif isSeq(p)       : b=interpret(p[1:],interpret(p[0],b))
    elif isZeroing(p)   : b[p[0]]=0;
    elif isAssignment(p): b[p[0]]=b[p[2]]
    elif isIncrement(p) : b[p[0]]=b[p[0]]+1
    elif isConditional(p):
        if b[p[1][0]]<b[p[1][2]] :  b=interpret(p[2],b)
        elif hasElse(p)          :  b=interpret(p[3],b)
    elif isLoop(p)      :  
        for _ in range(b[p[1]])  :  b = interpret(p[2],b) 
    elif isWhile(p) :
        if b[p[1][0]]<b[p[1][2]] : 
            b=interpret(p,interpret(p[2],b))
    else               : print(f'sollte nicht vorkommen P={p}')
    return b

... und auch Funktionen, um die Semantik eines Programms sowie die berechnete Funktion zu definieren

In [19]:
def semanticsWhile(p):
    def sem(b): return interpret(p,b)
    return sem

def f(P):
    tree=program.parseString(P, parseAll=True).asList()
    sem =semanticsWhile(tree)

    def fP(*args):
        args = [0]+list(args)
        beta={'x'+str(i): args[i] for i in range(len(args))}
        return sem(beta)['x0']
    
    return fP

In [20]:
prog_a_hoch_b = """x0++;
LOOP x2 DO 
    x101:=x0;
    x102:=x1;
    x0:=0;
    LOOP x102 DO 
        LOOP x101 DO
            x0++
        OD
    OD
OD"""
parse_tree = program.parseString(prog_a_hoch_b, parseAll=True).asList()
sem        = semanticsWhile(parse_tree)

und nun werten wir das Programm aus, drei mal, auf unterschiedlichen Wegen:

In [21]:
base=3
exponent=11

# ein mal über die interpret-Funktion
beta = {'x0':0,'x1':base,'x2':exponent}
print(f'Input: x1={beta["x1"]}, x2={beta["x2"]}')
beta=interpret(parse_tree, beta)
print("Result: "+str(beta['x0']))
print("Expected result:"+str(base**exponent)+ " equal? "+str(beta['x0']==(base**exponent)))

# ein mal über die Semantik
beta = {'x0':0,'x1':base,'x2':exponent}
beta=sem(beta)
print("Result: "+str(beta['x0']))
print("Expected result:"+str(base**exponent)+ " equal? "+str(beta['x0']==(base**exponent)))

# und schließlich über die berechnete Funktion:
pot=f(prog_a_hoch_b)
print("Result: "+str(pot(base,exponent)))
print("Expected result:"+str(base**exponent)+ " equal? "+str(beta['x0']==(base**exponent)))

Input: x1=3, x2=11
Result: 177147
Expected result:177147 equal? True
Result: 177147
Expected result:177147 equal? True
Result: 177147
Expected result:177147 equal? True


## Makros!
... gibt es in vielen Programmiersprachen für sich wiederholende Programmstücke.

So könnte man beispielsweise, wenn man öfter zu einer Variable 4 addieren muss:

`x2++;
x2++;
x2++;
x2++`

statt dessen `add4(x2)` schreiben. 

Wir erweitern dazu unseren WHILE-Interpreter um ein einfaches Makro-Konstrukt. Makros sind dabei nichts anderes als die Anwendung regulärer Ersetzungen auf das Programm. Falls Sie den folgenden Code nicht verstehen ist das nicht weiter schlimm. Wichtig ist, dass Sie weiter unten verstehen, wie man Makros definiert und in einem Programm verwendet.

In [53]:
import re

def apply_macro(m,p):
    if m[1]>0 :
        pattern = m[0] + "\\(" + "(x\\d*),"*(m[1]-1) + "(x\\d*)\\)"
    else :
        pattern = m[0] + "\\(\\)"
    return re.sub(pattern,m[2],p)

def apply_macros(ml, p) :
    q=p
    for m in ml:
        q=apply_macro(m,q)
    return q

Die Makros selbst sind (so wie wir sie definieren) Listen mit drei Elementen

`["Name", anzahl_argumente, "macro_code"]`

Das in der folgenden Zelle (s.u.) definierte Makro hat z.B. den Namen `add` und drei Argumente. Man kann es in einem Programm z.B. durch

`add(x1,x1,x5)`

aufrufen. Die Funktion `apply_macro(m,p)` ersetzt im Programm `p` dann jeden Makro-Aufruf von `m` durch den Code des Macros, wobei im Macro-Code die Platzhalter `\1, \2, ...` durch die betreffenden  Argumente ersetzt werden. Im Beipiel also hier z.B. `\1` durch `x1`, `\2` durch `x1` und `\3` durch `x5`. Beachten Sie, dass der String des `macro_code` undbedingt ein *raw*-String sein muss, also ein "r" vor dem String stehen muss.

In [54]:
m_add=["add",3, r"""\1:=\2;
LOOP \3 DO
\1++
OD"""]

Das folgende Programm veranschaulicht dies:

In [55]:
pmul="""x0:=0;
LOOP x2 DO 
add(x0,x0,x1)
OD"""

P=apply_macro(m_add,pmul)
print(f'Programm vor der Ersetzung:\n{pmul}\n\nProgramm nach der Makro-Ersetzung:\n{P}')

Programm vor der Ersetzung:
x0:=0;
LOOP x2 DO 
add(x0,x0,x1)
OD

Programm nach der Makro-Ersetzung:
x0:=0;
LOOP x2 DO 
x0:=x0;
LOOP x1 DO
x0++
OD
OD


Nun können wir zu etwas interessanteren Makros weiter schreiten und zunächst ein Multiplikationsmakro definieren. Damit dies auch für beliebige Wahl von `\1,\2 und \3` funktioniert benötigen wir zwei Hilfsvariablen, die wir im Bereich jenseits von `x100` ansiedeln.

In [56]:
m_mul=["mul",3,
       r"""x101:=\2;
        x102:=\3;
        \1:=0;
        LOOP x102 DO 
        add(\1,\1,x101)
        OD"""]

macros=[m_mul, m_add] # hier ist die Reihenfolge wichtig (m_mul verwendet m_add)

Nun definieren wir die Potenzfunktion auf Basis des Multiplikations:

In [57]:
ppot="""x0:=0;
x0++;
LOOP x2 DO 
    mul(x0,x0,x1)
OD"""
Q=apply_macros(macros, ppot)

Bestimmen die berechnete Funktion und werten sie an einem Beispiel aus.

In [58]:
pot=f(Q)
result = pot(2,10)
print(result)

1024


### Erweiterung von WHILE um andere Schleifen-Köpfe
In der folgenden Zelle definieren wir ein Makro, um in WHILE-Programme eine WHILE-Schleife mit einem Vergleich $x_i \leq x_j$ zu ermöglichen.

In [64]:
# Makro für WHILE xi <= xj
m = ["while_kg",2,r"""
x101:=\2;
x101++;
WHILE \1 < x101 
"""]

# Ein kleines Testprogramm (gibt x2-x1+1 aus)
prog="""
x0:=0;
while_kg(x1,x2) DO
   x1++;
   x0++
OD
"""

# Makro anwenden und Ergebnis ausgeben
prog=apply_macro(m,prog)
print(prog)

# Programm testen
fP=f(prog)
fP(3,7)


x0:=0;

x101:=x2;
x101++;
WHILE x1 < x101 
 DO
   x1++;
   x0++
OD



5